# Langchain 1.0

- langchain 1.0 실습 가이드입니다.
- 0버전과 문법 및 객체 구조가 매우 다르므로 GPT를 통한 코드 작성이 매우 어렵습니다. 반드시 작동이 확인된 예시 코드와 공식 문서를 기반으로 개발하시기 바랍니다.
- 코드 흐름을 따라가면서 이해되지 않는 개념이 있으면 함께 드린 마크다운 파일을 다시 읽어보세요!

## 1. 환경 설정

- Python 3.11.14 기반의 conda 가상 환경에서 테스트되었습니다.
- environment.yml 또는 requirements.txt 파일을 통해 환경을 재현하실 수 있습니다.
- 환경 변수만 잘 등록해준다면 venv, colab 등 다른 실행 환경에서도 작동할 것으로 예상되나 버전 정보를 잘 확인해주세요.
- langchain, langchain-core등 핵심 라이브러리는 꼭 1.0.x 버전을 사용해주세요.


In [ ]:
# 공통 라이브러리 import
import os

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

# 예쁜 출력
from pprint import pprint

## 2. 단일 Model 호출

- Agent 없이 단일 Model을 호출하는 방법을 알아봅니다.
- invoke()는 모델 또는 체인을 한 번 실행하여 모델의 응답을 반환받는 메서드입니다.
- Agent와 체인을 만들기 전에, 단일 Model을 호출해봅시다.

In [ ]:
# OpenAI 채팅 모델 초기화의 가장 기본적인 형태
from langchain.chat_models import init_chat_model
model = init_chat_model("openai:gpt-5-nano") # api key는 환경 변수에서 자동으로 로드됩니다. api_key 매개변수로 직접 전달할 수도 있습니다.

In [4]:
# invoke 메서드로 독립적인 LLM API를 호출합니다.
response = model.invoke('안녕')

# 모델이 보내준 응답 전체를 출력하면, 토큰 사용량 등 메타 데이터가 함께 출력됩니다.
pprint(response)

AIMessage(content='안녕하세요! 반가워요. 무엇을 도와드릴까요?\n\n원하실 때 골라서 시작해도 좋고, 그냥 대화로 시작해도 좋아요.\n- 간단한 정보 검색/설명\n- 글 작성, 번역, 첨삭\n- 아이디어 브레인스토밍\n- 코딩이나 수학 문제 풀이\n- 일상 대화 연습\n- 일정 계획이나 공부 계획 세우기\n\n다음 중에 원하시는 게 있나요? 아니면 다른 주제로 이야기해 봐도 좋습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 578, 'prompt_tokens': 8, 'total_tokens': 586, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWBVDftGflUJC0accVzMn4SQdw92L', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--bc3b0eae-c85d-46c3-b75b-30a0bb6b5f16-0', usage_metadata={'input_tokens': 8, 'output_tokens': 578, 'total_tokens': 586, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'

In [5]:
# 모델이 보내준 실제 답변 내용만 출력하려면 응답의 content 속성을 참조합니다.
pprint(response.content)

('안녕하세요! 반가워요. 무엇을 도와드릴까요?\n'
 '\n'
 '원하실 때 골라서 시작해도 좋고, 그냥 대화로 시작해도 좋아요.\n'
 '- 간단한 정보 검색/설명\n'
 '- 글 작성, 번역, 첨삭\n'
 '- 아이디어 브레인스토밍\n'
 '- 코딩이나 수학 문제 풀이\n'
 '- 일상 대화 연습\n'
 '- 일정 계획이나 공부 계획 세우기\n'
 '\n'
 '다음 중에 원하시는 게 있나요? 아니면 다른 주제로 이야기해 봐도 좋습니다.')


In [16]:
# 모델 초기화 시 다양한 파라미터를 지정할 수 있습니다.
# OpenAI 모델 외 다양한 모델에 대해서도 동일한 파라미터로 요청을 보낼 수 있습니다.
model = init_chat_model(
    model = "openai:gpt-5-nano",
    temperature=0.8, # 응답의 창의성. GPT-5 계열 모델들은 temperature 파라미터를 지원하지 않지만, Langchain에서는 일관된 인터페이스를 제공하기 때문에 오류가 발생하지 않습니다.
    max_tokens=2000, # 최대 토큰 수
    timeout=20, # 요청 제한 시간(초)
    max_retries=2, # 요청 실패 시 재시도 횟수
    )

In [ ]:
# 파라미터가 적용된 모델로도 다시 호출해봅시다.
response = model.invoke('너 웹 서치 기능도 가지고 있니?') # 기본 모델은 웹 서치 기능을 가지고 있지 않습니다.
pprint(response.content)

('기본적으로는 실시간 웹 검색을 자동으로 수행하지는 않아요. 다만 사용하는 인터페이스에 따라 웹 브라우징 기능이 활성화되어 있으면 최신 '
 '정보를 찾아 출처와 함께 제공할 수 있습니다.\n'
 '\n'
 '웹 검색을 원하시면 "웹 검색 기능으로 찾아줘"라고 요청해 주세요. 가능 여부를 확인한 뒤 최신 정보를 찾아 드리겠습니다.  \n'
 '그렇지 않으면 제 지식 범위(2024년 6월까지) 안에서 최대한 정확하게 답변해 드릴게요.\n'
 '\n'
 '참고로 검색이 가능하더라도 정보의 정확성을 항상 확인하는 것이 좋으니, 필요하신 경우 검색 쿼리도 함께 도와드립니다. 원하시는 주제나 '
 '키워드를 알려주실래요?')


## 3. Agent 생성

- 아무 추가 기능 없는 단일 모델 호출하려고 Langchain을 쓰는 건 아니죠.
- Langchain의 핵심 기능인 Agent를 만들어보겠습니다.

In [ ]:
# Agent 생성에 필요한 모듈입니다.
from langchain.agents import create_agent

# 앞서 초기화한 Model을 사용하여 Agent를 생성합니다.
agent = create_agent(
    model=model, # 사용할 LLM 모델을 미리 초기화(init)한 후 인자로 전달하세요.
    tools=[] # tool은 필수 인자지만, 빈 리스트를 전달하면 아무 tool도 없는 Agent를 생성할 수 있습니다.
    )

In [120]:
# 모델에 전달할 메시지를 OpenAI의 Chat API 형식에 맞게 작성합니다.
# 다른 모델은 다른 형식을 요구할 수 있으니, 모델 문서를 참고하세요.
messages = [
    {"role": "user", "content": "오늘 서울 날씨는 어때?"},
]

# Agent를 통해 모델을 호출합니다. Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
# 검색 tool이 없기 때문에, 실시간 날씨 정보를 제공하지는 못할 것입니다.
response = agent.invoke({"messages":messages})
pprint(response)

{'messages': [HumanMessage(content='오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='196d18d9-3f6d-4dbe-ad77-2ad43369a957'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 197, 'total_tokens': 354, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWFOMxSJFXiWJ5clZv5fdk0ipet39', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f2377e44-bf7b-40ce-82e3-4f4d304624ac-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '서울 날씨 오늘'}, 'id': 'call_Vtdc6iFsn6bEOqx3zE09dLL7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 197, 'output_tokens': 157, 'total_t

In [121]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('지금 바로 실시간으로 조회해서 알려드릴까요?  \n'
 '원하시면 제공할 정보 선택해 주세요:\n'
 '- 현재 기온·체감온도·강수(비/눈) 여부\n'
 '- 오늘 시간대별 예보(오전/오후/밤)\n'
 '- 강수 확률·강수량\n'
 '- 풍속·습도\n'
 '- 미세먼지·초미세먼지 등 대기질\n'
 '원하시는 항목 말해주시면 바로 확인해서 알려드릴게요.')


In [52]:
# Agent 생성 시에도 다양한 파라미터를 지정할 수 있습니다.
# 오늘 다 다뤄보지는 못하겠지만, Agent의 세부 동작은 모두 이 파라미터들로 조정할 수 있습니다.
# 아래 있는 것들 외에도 더 많은 파라미터가 있으니 공식 문서를 참고하세요.
agent = create_agent(
    model=model,
    tools=[], # tool 리스트는 다음 챕터에서 다뤄보겠습니다.
    system_prompt="You are a helpful assistant.", # Agent의 시스템 프롬프트입니다.
    middleware=[], # Agent의 동작 사이에 발생하는 요청이나 응답을 가로채서 처리하는 고급 기능입니다.
)

## 3. tool 정의

- 위에서 만든 Agent는 별다른 기능이 없으니 단일 모델 호출과 별로 다를 게 없죠.
- 이제 Agent가 사용할 tool을 만들어 봅시다.
- tool의 종류를 세 가지로 나누어 각각 만들어보겠습니다.

### 3-1. 커스텀 tool 정의

- 직접 만든 Python 함수를 tool로 사용할 수 있습니다.
- URL 또는 라이브러리를 통해 API를 호출하여 응답을 반환하는 tool도 이 방법으로 구현할 수 있습니다.

In [53]:
# tool 정의를 위해 필요한 모듈입니다.
from langchain.tools import tool

# 커스텀 tool 생성
@tool # 데코레이터를 사용하여 tool로 등록합니다.
def calculator(num_1:int, num_2:int) -> int: # typehint는 Agent가 tool의 입출력 형식을 이해하는 데 도움을 줍니다. 안정적인 작동을 위해 반드시 작성하는게 좋습니다.
    """입력받은 두 수의 덧셈을 반환합니다.""" # docstring은 tool의 설명으로 사용됩니다. Agent가 tool을 선택하는 데 도움을 줍니다.
    return num_1 + num_2

### 3-2. langchain-community에 통합된 외부 tool 사용

- 라이브러리를 통해 제공되는 외부 tool들을 langchain-community에서 통합하여 사용할 수 있습니다.
- 웬만한 기능들은 langchain 생태계 내에서 쉽게 쓸 수 있으니 목록을 확인하고 활용해봅시다.
- https://docs.langchain.com/oss/python/integrations/providers/overview

In [ ]:
# langchain community에서 통합되어 있는 외부 tool의 목록을 확인해봅시다.
import importlib, pkgutil

package = importlib.import_module("langchain_community.tools")

for mod in pkgutil.iter_modules(package.__path__):
    print(mod.name)

ainetwork
amadeus
arxiv
asknews
audio
azure_ai_services
azure_cognitive_services
bearly
bing_search
brave_search
cassandra_database
clickup
cogniswitch
connery
convert_to_openai
databricks
dataforseo_api_search
dataherald
ddg_search
e2b_data_analysis
edenai
eleven_labs
few_shot
file_management
financial_datasets
github
gitlab
gmail
golden_query
google_books
google_cloud
google_finance
google_jobs
google_lens
google_places
google_scholar
google_search
google_serper
google_trends
graphql
human
ifttt
interaction
jina_search
jira
json
memorize
merriam_webster
metaphor_search
mojeek_search
multion
nasa
nuclia
office365
openai_dalle_image_generation
openapi
openweathermap
passio_nutrition_ai
playwright
plugin
polygon
powerbi
pubmed
render
requests
riza
scenexplain
searchapi
searx_search
semanticscholar
shell
slack
sleep
spark_sql
sql_database
stackexchange
steam
steamship_image_generation
tavily_search
vectorstore
wikidata
wikipedia
wolfram_alpha
yahoo_finance_news
you
youtube
zapier
zenguar

In [54]:
# duckduckgo는 api key 없이도 사용할 수 있는 웹 검색 도구입니다.
import langchain_community.tools.ddg_search

# duckduckgo 검색 tool의 속성을 확인하여 검색 기능 호출 방법을 알아봅시다.
print(dir(langchain_community.tools.ddg_search))

['DuckDuckGoSearchRun', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'tool']


In [32]:
# 위에서 확인한 duckduckgo의 검색 기능을 import합니다.
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

# 검색 tool 인스턴스를 생성합니다.
ddg_search_tool = DuckDuckGoSearchRun()

In [ ]:
# 세 번째 방법을 배우기 전에, 위에서 만든 두 가지 tool을 Agent에게 전달해봅시다.
# nano 모델은 Agent로 사용하기엔 너무 성능이 낮아 무한 루프 등 의도하지 않은 동작이 발생할 수 있으므로 mini 모델로 변경하겠습니다.
model = init_chat_model("openai:gpt-5-mini")

# agent 생성 및 tool 전달
agent = create_agent(
    model=model,
    tools=[calculator, ddg_search_tool], # 앞서 만든 두 가지 tool을 리스트로 전달합니다.
)

In [ ]:
# Agent에 전달할 메시지 객체를 생성하기 위해 필요한 모듈입니다.
from langchain.messages import HumanMessage

# Agent에 전달할 메시지 객체를 생성합니다.
messages = [
    HumanMessage(content="2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?"),
]

# Agent에 대한 invoke는 모델 invoke와는 다르게 dictionary 형태의 입력을 받으니 주의하세요.
response = agent.invoke({"messages":messages})


# Agent가 보내준 전체 응답에는 추론 과정과 tool 사용 내역이 포함되어 있습니다.
pprint(response)

{'messages': [HumanMessage(content='2 더하기 3은 얼마야? 그리고 오늘 서울 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='7208118b-2a6e-4bc8-9dd2-b88204350631'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 259, 'prompt_tokens': 207, 'total_tokens': 466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWDoMAhw7TxzBC1vdMs6octKU1qyM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--d585fce1-9d0b-49db-a35e-84618fd1d950-0', tool_calls=[{'name': 'calculator', 'args': {'num_1': 2, 'num_2': 3}, 'id': 'call_0GXxLFwcKTYRgkN4WJt6MwF8', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': '서울 날씨

In [78]:
# 실제 답변 내용만 출력하려면 messages 리스트의 마지막 요소의 content 속성을 참조합니다.
pprint(response["messages"][-1].content)

('2 + 3 = 5.\n'
 '\n'
 '오늘 서울 날씨는 지금 바로 최신 정보를 다시 확인해 드릴게요. 방금 검색 결과는 날짜가 섞여 있어 정확하지 않습니다(예: 과거의 '
 '“부분적으로 흐림, 32°C” 같은 정보가 섞여 나옵니다). 지금 최신 기온·강수확률·미세먼지 등을 알려드릴까요? 원하시면 바로 확인해서 '
 '알려드릴게요.')


### 3-3. 모델 vendor들이 제공하는 tool을 모델에 바인딩

#### **※ 주의!! ※**
- 현재 버전에서 Agent를 통하지 않고 단일 Model을 직접 호출해야만 사용할 수 있는 기능입니다.
- Agent를 활용하는 경우 사용할 수 없는 방법이므로 헷갈리지 마세요!
- 이 방법을 아예 모르고 있으면 GPT나 Docs 보고 코드 쓰다가 tool 정의 및 호출 방법을 섞어서 쓰다가 코드가 꼬일 수 있기 때문에 개념만 이해하고 넘어갑시다.
---
#### 실습
- LLM API를 제공하는 vendor들은 모델에서 곧바로 이용할 수 있는 tool을 함께 서비스하고 있습니다.
- 예를 들어, OpenAI의 경우 웹 서치, MCP 서버 연결, 벡터 스토어 검색, 코드 실행 등의 도구를 제공합니다.
- vendor가 제공하는 tool을 Agent에 바인딩하여 사용해보겠습니다.

In [106]:
from langchain_openai import ChatOpenAI

# 객체 생성
llm = ChatOpenAI(
    temperature=0.5,
    model_name="gpt-5-mini",  # 모델명
)

# 질의내용
question = "대한민국의 수도는 어디인가요?"

# 질의
print(f"[답변]: {llm.invoke(question)}")

[답변]: content='대한민국의 수도는 서울특별시(서울)입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 215, 'prompt_tokens': 15, 'total_tokens': 230, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CWEixUYzsFSqqvXRNzcgMWUJVSA6y', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--fb9bf966-4dd6-4a16-baf7-ea0485896644-0' usage_metadata={'input_tokens': 15, 'output_tokens': 215, 'total_tokens': 230, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}


In [115]:
# 모델 vendor들이 제공하는 tool을 모델에 바인딩하기 위해 필요한 모듈입니다.
from langchain_openai import ChatOpenAI

# OpenAI 전용 Model 초기화 메서드입니다.
# OpenAI 모델에서 제공하는 tool을 바인딩하기 위해서는 모델 초기화시 ChatOpenAI 클래스를 사용해야 합니다.
model = ChatOpenAI(
    model="gpt-5-mini",
)

# ChatOpenAI 모델 인스턴스에 커스텀 tool을 바인딩합니다.
tools = model.bind_tools([calculator]) # openai가 제공하는 검색 tool을 사용하므로 ddg_search_tool은 제외합니다.

# openai가 제공하는 웹 검색 tool을 정의합니다.
openai_tool = [{"type": "web_search"},]

# model을 호출합니다.
response = model.invoke(
    "올해 11월에 한국에서 개봉하는 영화 알려줘. 그리고 11 더하기 15는 몇이야?",
    tools=openai_tool # openai가 제공하는 tool 중 사용할 것들을 리스트로 전달합니다.
)

pprint(response)

AIMessage(content=[{'id': 'rs_0eecc8b18cdac91f006902f08059748196ad112fac4ed126fe', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0eecc8b18cdac91f006902f08316d88196a5ccddbdcbf469f0', 'action': {'query': '2025년 11월 한국 개봉 영화 목록', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0eecc8b18cdac91f006902f0859cf8819696e2255719edb98b', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0eecc8b18cdac91f006902f087f504819697ea4ea3fb486686', 'action': {'query': '위키드 포 굿 한국 개봉일 2025 11월', 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0eecc8b18cdac91f006902f08a81bc8196a42207f1e9b4fbdc', 'summary': [], 'type': 'reasoning'}, {'id': 'ws_0eecc8b18cdac91f006902f08bacb88196bf0ddfc98f539707', 'action': {'query': "Universal Pictures Korea Wicked: For Good 한국 개봉일 유니버설 픽쳐스 코리아 '위키드: 포 굿' 개봉일", 'type': 'search'}, 'status': 'completed', 'type': 'web_search_call'}, {'id': 'rs_0eecc8b18cdac91f006902f08e5a74819697c3fb888a8d4209', 'summary': [], 'type

In [116]:
# 이번에도 실제 답변 내용만 출력해봅시다. 조금 더 깔끔한 출력을 위해 content의 각 파트를 순회하며 줄 단위로 출력합니다.
for part in response.content:
    if part.get("type") == "text":
        print(part["text"])

좋습니다 — 기준을 분명히 할게요. 제가 말하는 “올해 11월”은 2025년 11월을 의미합니다(오늘 기준: 2025‑10‑30). 아래는 2025년 11월에 한국에서 개봉(또는 한국 관객에게 공개 예정)하는 주요 기대작들입니다.

주요 작품(일시·간단 정보)
- 위키드: 포 굿 (Wicked: For Good) — 한국: 2025년 11월 19일 전세계 최초 개봉(수입·배급: 유니버설 픽쳐스). ([nocutnews.co.kr](https://www.nocutnews.co.kr/news/6405746?utm_source=openai))  
- 나우 유 씨 미 3 (Now You See Me 3) — 한국: 2025년 11월 12일 개봉(배급: 롯데엔터테인먼트). ([news.tf.co.kr](https://news.tf.co.kr/read/entertain/2252156.htm?utm_source=openai))  
- 주토피아 2 (Zootopia 2) — 한국: 2025년 11월 개봉 예정(북미 기준 2025‑11‑26; 디즈니코리아는 국내에 11월 개봉을 발표). ([koreafilm.co.kr](https://www.koreafilm.co.kr/movie/review/zootopia2.htm?utm_source=openai))  
- 프랑켄슈타인 (Guillermo del Toro’s Frankenstein) — 넷플릭스 공개일: 2025년 11월 7일(넷플릭스 공개 기준, 일부 지역 극장 개봉 일정과 다름). (한국에서 스트리밍 공개는 넷플릭스 제공 일정 적용). ([sortiraparis.com](https://www.sortiraparis.com/ko/palieseo-hal-il/yeonghwa-silijeu-chugje/articles/335739-yeonghwa-peulangkensyutain-2025?utm_source=openai))

참고/추가
- 위에 올린 건 주요 기대작 위주입니다. 11월에는 독립·예술영화나 일부 외화의 국내 개봉일

## 4. 대화 기록 저장
